# STAGE III - REAL WORLD OPTIMIZATION

![FIGURE](STAGE-III.png)

## DEPENDENCIES

In [1]:
# COMMON
import numpy as np
import sys
import os
import random
import time
import datetime
import pandas as pd
import matplotlib.pyplot as plt

# Triggering Mechanism
import triggering as trig

# Signal Processing
from scipy import signal  # for signal processing
from scipy.signal import hilbert  # for signal processing
import prenn # for deep learning

# AI
import keras

import tensorflow as tf # for deep learning
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import TensorBoard # for visualization

import onnxruntime as ort # for onnx runtime

# Visualization
# %matplotlib qt5
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

## CLOSED-LOOP CONTROL COMPONENTS SETUP

### ENVIRONMENT

IN NATURE - THE SYNTHESIZED RESPONSE DATA - DIFFERENT FROM STAGE II

In [2]:
# Corrected Paths
AVPath = r'../01-PRE-DEPLOYMENT/DATA/VAL_RES/VAL_DATA_AV.npy' 
EQPath = r'../01-PRE-DEPLOYMENT/DATA/VAL_RES/VAL_DATA_EQ.npy'
IPPath = r'../01-PRE-DEPLOYMENT/DATA/VAL_RES/VAL_DATA_IP.npy'
SWPath = r'../01-PRE-DEPLOYMENT/DATA/VAL_RES/VAL_DATA_SW.npy'

# Load Data
RESPONSE_AV = np.load(AVPath)
RESPONSE_EQ = np.load(EQPath)
RESPONSE_IP = np.load(IPPath)
RESPONSE_SW = np.load(SWPath)

## only use part of the data
raw_len_av = RESPONSE_AV.shape[0]
raw_len_eq = RESPONSE_EQ.shape[0]
raw_len_ip = RESPONSE_IP.shape[0]
raw_len_sw = RESPONSE_SW.shape[0]

# this is to simulate the imbalanced data - ensure the same ratio within STAGE III
ratio_aw = 0.6
ratio_eq = 1
ratio_ip = 1
ratio_sw = 1

uselen_aw = int(raw_len_av * ratio_aw)
uselen_eq = int(raw_len_eq * ratio_eq)  
uselen_ip = int(raw_len_ip * ratio_ip)
uselen_sw = int(raw_len_sw * ratio_sw)

# randomly pick uselen_aw samples from RESPONSE_XX, no repeat
idx_aw = random.sample(range(raw_len_av), uselen_aw)
idx_eq = random.sample(range(raw_len_eq), uselen_eq)
idx_ip = random.sample(range(raw_len_ip), uselen_ip)
idx_sw = random.sample(range(raw_len_sw), uselen_sw)

RESPONSE_AV = RESPONSE_AV[idx_aw]
RESPONSE_EQ = RESPONSE_EQ[idx_eq]
RESPONSE_IP = RESPONSE_IP[idx_ip]
RESPONSE_SW = RESPONSE_SW[idx_sw]

## Check Data Info
print(type(RESPONSE_AV))
print('RESPONSE_AV Shape:', RESPONSE_AV.shape)
print(type(RESPONSE_EQ))
print('RESPONSE_EQ Shape:', RESPONSE_EQ.shape)
print(type(RESPONSE_IP))
print('RESPONSE_IP Shape:', RESPONSE_IP.shape)
print(type(RESPONSE_SW))
print('RESPONSE_SW Shape:', RESPONSE_SW.shape)

num_type = 4

signal_length = RESPONSE_AV.shape[1]

LenTS = signal_length

# ratio of non-interested data to interested data
ratio_ni = RESPONSE_AV.shape[0] / (RESPONSE_EQ.shape[0] + RESPONSE_IP.shape[0] + RESPONSE_SW.shape[0])

print('Ratio of Non-Interested Data to Interested Data:', ratio_ni)

## Stack Data
RESPONSE = np.vstack((RESPONSE_AV, RESPONSE_EQ, RESPONSE_IP, RESPONSE_SW))

## Total Number of Samples
NumSample = RESPONSE.shape[0]
print('Total Number of Samples:', NumSample) 

<class 'numpy.ndarray'>
RESPONSE_AV Shape: (150, 6000)
<class 'numpy.ndarray'>
RESPONSE_EQ Shape: (50, 6000)
<class 'numpy.ndarray'>
RESPONSE_IP Shape: (50, 6000)
<class 'numpy.ndarray'>
RESPONSE_SW Shape: (50, 6000)
Ratio of Non-Interested Data to Interested Data: 1.0
Total Number of Samples: 300


In [3]:
NUM_AV = RESPONSE_AV.shape[0]
NUM_EQ = RESPONSE_EQ.shape[0]
NUM_IP = RESPONSE_IP.shape[0]
NUM_SW = RESPONSE_SW.shape[0]

#print
print('Number of AV:', NUM_AV)
print('Number of EQ:', NUM_EQ)
print('Number of IP:', NUM_IP)
print('Number of SW:', NUM_SW)

Number of AV: 150
Number of EQ: 50
Number of IP: 50
Number of SW: 50


### SYSTEM
IN NATURE - THE TRIGGERING MECHANISM

In [4]:
# embodied in the triggering.py

In [5]:
# determine the search space by check the maximum and minimum of the data - ambient vibration
max_noise = np.max(abs(RESPONSE_AV))
min_noise = np.min(abs(RESPONSE_AV))
print('Maximum of AV:', max_noise)
print('Minimum of AV:', min_noise)

# determine the lower and upper bound of the search space
ub_factor = 2
lb_factor = 0.01

Maximum of AV: 0.0168420367172138
Minimum of AV: 5.263614397604116e-06


In [6]:
# configuration

## para_a stands for the triggering threshold amplitude (continuous)
para_a_lb = lb_factor*(max_noise + min_noise)/2
para_a_ub = ub_factor*(max_noise + min_noise)/2

print('Lower Bound of para_a:', para_a_lb)
print('Upper Bound of para_a:', para_a_ub)

## para_b stands for the triggering threshold duration (discrete integer)
para_b_lb = 2
para_b_ub = 10

print('Lower Bound of para_b:', para_b_lb)
print('Upper Bound of para_b:', para_b_ub)

Lower Bound of para_a: 8.423650165805703e-05
Upper Bound of para_a: 0.016847300331611404
Lower Bound of para_b: 2
Upper Bound of para_b: 10


In [7]:
# test
test_data = RESPONSE_AV[0]

para_a = 0.5
para_b = 2

trig_flg = trig.activation(test_data, para_a, para_b)
print(trig_flg[0]) # the first element stands for the triggering flag, and the second element stands for the triggering time

0


### ESTIMATOR
IN NATURE - TRIGGERING MECHANISM & GROUND TRUTH

In [8]:
beta = 1.75 # beta value to balance the precision and recall

In [9]:
# USED PARAMETER of the Dataset

# nn_in_len = 128 # nn input length can be automatically calculated from the model input size
dt = 0.01  # time step, change according to the dataset
nperseg = 128 # number of points for each segment in spectrogram

#### Classifier - CNN Model

In [10]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [11]:
# CNN Classifier Model Path
classifier_model_path = r'../01-PRE-DEPLOYMENT/03-NN-TRAINING-CNN/cnn_model.keras'
CNN_Model = keras.saving.load_model(classifier_model_path)
CNN_Model.summary()

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 128, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv1D', 'config': {'name': 'conv1d', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 4, 'kernel_size': [3], 'strides': [1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1], 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'see

Predictor - DNN Model

In [ ]:
# DNN Predictor Model Path
recall_model_path = r'../01-PRE-DEPLOYMENT/04-NN-TRAINING-DNN/recall_est.keras'
Recall_Model = keras.saving.load_model(recall_model_path)
Recall_Model.summary()

# DNN Predictor Model Path
precision_model_path = r'../01-PRE-DEPLOYMENT/04-NN-TRAINING-DNN/precision_est.keras'
Precision_Model = keras.saving.load_model(precision_model_path)
Precision_Model.summary()

In [ ]:
# get the input data length
nn_in_len = CNN_Model.input_shape[1]
print('Input Data Length:', nn_in_len)

In [ ]:
# model test by loading a piece of data from the dataset, and feed intot he prenn function

# index for the random data
idx = np.random.randint(NumSample)

# get the data
test_data = RESPONSE[idx, :].reshape(1, LenTS)

print('Test Data Shape:', test_data.shape)

# plot the test data
plt.figure()
plt.plot(test_data[0, :])
plt.title('Test Data')
plt.show()


In [ ]:
# preprocess
output_length = nn_in_len
test_data = prenn.prenn(test_data, dt, nperseg, output_length)

# print the preprocessed data
plt.figure()
plt.plot(test_data[0, :])
plt.title('Preprocessed Test Data')
plt.show()

In [ ]:
test_data = test_data.reshape(1, nn_in_len, 1)

print('Preprocessed Test Data Shape:', test_data.shape)

# test the model
test_result = CNN_Model.predict(test_data)
print('Tensorflow Model Prediction:', test_result)

# parse the result to get the label
result_label = np.argmax(test_result)

print('Tensorflow Model Prediction:', result_label)

### CONTROLLER
IN NATURE - THE BAYESIAN OPTIMIZATION FRAMEWORK FOR OPTIMIZATION AND CONTROLL

In [ ]:
# the bayesian optimization algorithm, will be covered in the next section

## BAYESIAN OPTIMIZATION FRAMEWORK

### Helper Functions

In [ ]:
# %pip install PyQt5
# %pip install colorama


In [ ]:
# # use pop-up window for plots
# %matplotlib qt5 
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib.animation import FuncAnimation
# # print control
# from colorama import init, Fore, Back, Style

In [ ]:
# CONFIGURABLE PARAMETERS

## Hyperparameters
noise_lvl = 0.1
k_alpha = 1
k_lambda = 1

## Process Control
num_init_points = 15
num_iter = 20
tolerance = 1e-3
gap = np.inf

## Acquisition Function - UCB
ucb_beta = 1

## Acquisition Function - PI & EI
xi=0.01

## Bonus factor for evaluation
bonus_factor = 1.1

In [ ]:
## Search Space
### parameter 1 - a
lb_a = para_a_lb
print('Lower Bound of para_a:', lb_a)
ub_a = para_a_ub
print('Upper Bound of para_a:', ub_a)
c_a = (ub_a + lb_a) / 2

### parameter 2 - b
lb_b = para_b_lb
print('Lower Bound of para_b:', lb_b)
ub_b = para_b_ub
print('Upper Bound of para_b:', ub_b)
c_b = (ub_b + lb_b) / 2
print(c_b)

### The Problem / System to Optimize

Triggering Mechanism： Triggering Parameters In; Evaluation Index Out

In [ ]:
# THE OBJECTIVE FUNCTION
def objective_function(para_a, para_b):

    """ Function with unknown internals we wish to maximize.

    This function defines the process to evaluate how good the parameters are for the triggering sensing mechanism.

    INPUT:
    - para_a: the threshold for the activation, positive real number
    - para_b: activate_duration: the duration for the activation, positive integer >= 2 (as we know 1 is not error-prone in practice)

    OUTPUT:
    - evaluation value: F-beta score
    
    HYPERPARAMETERS:
    - beta: the value to balance the precision and recall to calculate F-beta score
    
    """
    
    # assistive variables
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    
    # go through the dataset
    for i in range(NumSample):
    # for i in range(10):  
        # trigger flag
        flg_trigger = 0
        
        # ground truth flag
        class_1h = np.zeros((1, num_type))
        flg_class = 0
        flg_interst = 0
        
        # get the response data
        signal = RESPONSE[i, :].reshape(1, LenTS)
        
        # get the triggering mechanism
        flg_trigger, trig_pos = trig.activation(signal, para_a, para_b)
        
        # print('Trigger Flag:', flg_trigger)
        
        # preprocess the signal for the CNN model to get the ground truth
        signal = prenn.prenn(signal, dt, nperseg, nn_in_len)
        
        signal = signal.reshape(1, nn_in_len, 1)

        # get the ground truth
        class_1h = CNN_Model.predict(signal, verbose=0)
        flag_class = np.argmax(class_1h)
        
        # get the ground truth
        if flag_class > 0:
            flg_interst = 1

        # get the confusion matrix - as the number of interested data is assumed to be small, we need to consider the ratio - only two classes - interested and non-interested
        if flg_trigger == 1 and flg_interst == 1:
            TP += 1
        elif flg_trigger == 1 and flg_interst == 0:
            FP += 1
        elif flg_trigger == 0 and flg_interst == 1:
            FN = -1 # no use, recall will be calculated in other way
        elif flg_trigger == 0 and flg_interst == 0:
            TN = -1 # no use, recall will be calculated in other way
        else:
            print('Error in Confusion Matrix Calculation')
    
    # recall = Precision_Model
    std_av = np.std(RESPONSE_AV[0, :])
    recall_input = np.array([std_av, para_a, para_b]).reshape(1, 3)
    recall = Recall_Model.predict(recall_input)
    recall = float(recall)
    
    if recall > 1:
        recall = 1
    if recall < 0:
        recall = 0
    
    # calculate precision
    precision = TP / (TP + FP)
            
    # calculate the F-beta score
    denominator = (beta**2) * precision + recall 
    if denominator == 0:
        F_beta = 0
    else:
        F_beta = (1 + beta**2) * precision * recall / denominator
    
    # if F_beta is NaN, return 0
    if np.isnan(F_beta):
        F_beta = 0
        return 0
    
    if precision > 0.9 and recall > 0.9:
        F_beta = F_beta * bonus_factor

    
    
    # print TP, FP, FN, TN as integer, Precision, Recall, F-beta as float number with 4 decimal places
    # print('para_a / para_b /TP / FP / FN / TN / Precision / Recall / F-beta:', para_a, para_b, TP, FP, FN, TN, f'{precision:.4f}', f'{recall:.4f}', f'{F_beta:.4f}')  
    # print('para_a / para_b ', para_a, para_b)  
    # print('TP / FP / FN / TN / Precision / Recall / F-beta:', TP, FP, FN, TN)  
    # print('Precision / Recall / F-beta:', f'{precision:.4f}', f'{recall:.4f}', f'{F_beta:.4f}')  
    print(f'para_a / para_b / TP / FP / FN / TN / Precision / Recall / F-beta: {para_a} {para_b} {int(TP)} {int(FP)} {int(FN)} {int(TN)} {precision:.4f} {recall:.4f} {F_beta:.4f}')

    return F_beta, TP, FP, FN, TN, precision, recall


In [ ]:
# THE SURROGATE MODEL - Gaussian Process - Kernel Function
## Matern 2.5 as the kernel function
def kernel(p1, p2, matern_alpha = 1, matern_lambda = 1):
    # define the distance between two points p1 and p2
    d= np.linalg.norm(p1 - p2)
    k = matern_alpha * (1 + np.sqrt(5) * d / matern_lambda + 5 * d**2 / (3 * matern_lambda**2)) * np.exp(-np.sqrt(5) * d / matern_lambda)
    return k

## Mean and Variance of the Gaussian Process
def mean_var(x, D, K_alpha, k_lambda):
    # assume x is a variable of 1x1
    # calculate the mean and variance of the Gaussian Process
    x_dim = x.shape[1] # assume each colum is a dimension of x
    num_D = D.shape[0] # number of data points in D
    
    Ktt = np.zeros((num_D, num_D))
    for i in range(num_D):
        for j in range(num_D):
            Ktt[i, j] = kernel(D[i,:-1], D[j,:-1], K_alpha, k_lambda)
    Kttn = Ktt + np.eye(num_D) * noise_lvl
    IKttn = np.linalg.inv(Kttn)
    
    Kpt = np.zeros((1, num_D))
    for i in range(num_D):
        Kpt[0, i] = kernel(x, D[i,:-1], K_alpha, k_lambda)
    Ktp = Kpt.T
    Kpp = kernel(x, x, K_alpha, k_lambda)
    
    y = D[:,-1].reshape(-1,1)
    
    mean = Kpt @ IKttn @ y
    var = Kpp - Kpt @ IKttn @ Ktp
        
    return mean, var

In [ ]:
# THE ACQUISITION FUNCTION - UPPER CONFIDENCE BOUND
def S(x, D):
    mean, var = mean_var(x, D, k_alpha, k_lambda)
    s = mean + ucb_beta * np.sqrt(var)
    return s

In [ ]:
def norm_cdf(x):
    """
    standard normal cumulative distribution function
    
    """
    return 0.5 * (1 + np.math.erf(x / np.sqrt(2)))

In [ ]:
def norm_pdf(x):
    """
    standard normal probability density function
    """
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x ** 2)

In [ ]:
# THE ACQUISITION FUNCTION - PROBABILITY OF IMPROVEMENT
def S_PI(x, D, xi=0.01):
    f_best = np.max(D[:,-1])
    mean, var = mean_var(x, D, k_alpha, k_lambda)
    z = (mean - f_best - xi) / np.sqrt(var)
    s = norm_cdf(z)
    return s

In [ ]:
# THE ACQUISITION FUNCTION - EXPECTATION OF IMPROVEMENT
def S_EI(x, D, xi=0.01):
    f_best = np.max(D[:,-1])
    mean, var = mean_var(x, D, k_alpha, k_lambda)
    z = (mean - f_best - xi) / np.sqrt(var)
    s = (mean - f_best - xi) * norm_cdf(z) + np.sqrt(var) * norm_pdf(z)
    return s

In [ ]:
# THE OPTIMIZATION FUNCTION FOR THE ACQUISITION FUNCTION
def optimize_acquisition(D):
    # randomly search for the point with the highest acquisition function value
    num_search = 100
    max_s = -np.inf
    max_x = None
    cur_x = np.zeros((1,2))
    for i in range(num_search):
        cur_x[0,0] = np.random.uniform(lb_a, ub_a)
        cur_x[0,1] = np.random.uniform(lb_b, ub_b)
        cur_s = S(cur_x, D) # UCB
        # cur_s = S_PI(cur_x, D, xi) # PI
        # cur_s = S_EI(cur_x, D, xi) # EI
        if cur_s > max_s:
            max_s = cur_s
            max_x = cur_x.copy()
    return max_x

### The Optimization Process

In [ ]:
# load data in STAGE II - D
D_Path = r"../01-PRE-DEPLOYMENT/05-PRE-OPTIMIZATION/D.npy"

D = np.load(D_Path)

stage_virtual_num = D.shape[0]

print('Number of Data Points in pre-optimization (D number):', stage_virtual_num)

DD_Path = r"../01-PRE-DEPLOYMENT/05-PRE-OPTIMIZATION/DD.npy"

DD = np.load(DD_Path)

print('Number of Data Points in pre-optimization (DD number):', DD.shape[0])

In [ ]:
max_val = np.max(D[:,2])
print('Initial Maximum Value:', max_val)

In [ ]:
com_a = 0.014
com_b = 5
comparison = objective_function(com_a, com_b)
com_f_beta = comparison[0]
com_recall = comparison[-2]
com_precision = comparison[-1]
print('Comparison F-beta:', com_f_beta)
print('Comparison Recall:', com_recall)
print('Comparison Precision:', com_precision)